In [2]:
import os
print(os.path.exists("haarcascade_russian_plate_number.xml"))


True


In [3]:
import os
print("Current Working Directory:", os.getcwd())  # Show where the notebook is running
print("Files in This Folder:", os.listdir())  # List all files


Current Working Directory: C:\NPD\ComputerVision-main\NumberPlateDetection
Files in This Folder: ['.ipynb_checkpoints', 'detected_plates.txt', 'haarcascade_russian_plate_number.xml', 'np.xml', 'NumberPlateDetection.ipynb', 'pla', 'plates']


In [4]:
import os
print("Files in Directory:", os.listdir())


Files in Directory: ['.ipynb_checkpoints', 'detected_plates.txt', 'haarcascade_russian_plate_number.xml', 'np.xml', 'NumberPlateDetection.ipynb', 'pla', 'plates']


In [5]:
import os
print("Current Working Directory:", os.getcwd())



Current Working Directory: C:\NPD\ComputerVision-main\NumberPlateDetection


In [6]:
import os
import cv2
import easyocr
import re
import time
import numpy as np
import tkinter as tk
from tkinter import filedialog, simpledialog

# Ensure output folders exist
output_folder = "plates"
os.makedirs(output_folder, exist_ok=True)

# Ensure detected_plates.txt exists
plate_file = "detected_plates.txt"
if not os.path.exists(plate_file):
    open(plate_file, "w").close()

# Load Haarcascade model for number plate detection
np_cascade = cv2.CascadeClassifier('np.xml')

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Function to extract license plate numbers
def extract_plate_number(text):
    if not text:
        return None

    # Define valid license plate patterns
    plate_patterns = [
        r'^[A-Z]{2}[0-9]{1,2}[A-Z]{1,2}[0-9]{4}$',  # KA02MM9091
        r'^[A-Z]{2}[0-9]{2} [0-9]{4}$',             # MH12 3456
        r'^[A-Z]{2} [0-9]{2} [A-Z]{1,2} [0-9]{4}$'  # UP 32 AB 1234
    ]

    for pattern in plate_patterns:
        match = re.findall(pattern, text)
        if match:
            return match[0]

    return None

# Function to detect number plates
def detect_number_plate(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    plates = np_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=3, minSize=(40, 40))

    for (x, y, w, h) in plates:
        roi = frame[y:y+h, x:x+w]
        result = reader.readtext(roi)

        text = " ".join([res[1] for res in result]).strip()
        plate_number = extract_plate_number(text)

        if plate_number:
            # Save Plate Image
            timestamp = int(time.time())
            filename = os.path.join(output_folder, f"plate_{timestamp}.jpg")
            cv2.imwrite(filename, roi)
            print(f"✅ Valid Plate Detected: {plate_number}")
            print(f"✅ Plate Image Saved: {filename}")

            # Save detected plate number to a file
            try:
                with open(plate_file, "a", encoding="utf-8") as file:
                    file.write(f"{plate_number} - {time.ctime(timestamp)} | Saved at: {filename}\n")
                    file.flush()  # Ensures immediate save
                print(f"✅ Successfully written to file: {plate_file}")
            except Exception as e:
                print(f"❌ Error Writing to File: {e}")
        else:
            print("⚠ Detected but not a number plate.")

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, "Number Plate" if plate_number else "Not a Number Plate", 
                    (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    return frame

# Function to process webcam
def process_webcam():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not access the webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = detect_number_plate(frame)

        cv2.imshow("Number Plate Detection - Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to process video
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open the video file.")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video.")
            break

        frame = detect_number_plate(frame)

        cv2.imshow("Number Plate Detection - Video", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to select option
def select_option():
    root = tk.Tk()
    root.withdraw()

    choice = simpledialog.askinteger("Choose Option", "Select Mode:\n1. Webcam\n2. Video File")

    if choice == 1:
        process_webcam()
    elif choice == 2:
        video_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video Files", "*.mp4;*.avi;*.mov")])
        if video_path:
            process_video(video_path)
    else:
        print("Invalid choice. Please select 1 or 2.")

# ✅ Run the Program
select_option()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


⚠ Detected but not a number plate.
⚠ Detected but not a number plate.
⚠ Detected but not a number plate.
✅ Valid Plate Detected: MH 19 EQ 0001
✅ Plate Image Saved: plates\plate_1742721755.jpg
✅ Successfully written to file: detected_plates.txt
✅ Valid Plate Detected: GJ 19 EQ 0001
✅ Plate Image Saved: plates\plate_1742721755.jpg
✅ Successfully written to file: detected_plates.txt
⚠ Detected but not a number plate.
⚠ Detected but not a number plate.
⚠ Detected but not a number plate.
✅ Valid Plate Detected: MH 19 EQ 0001
✅ Plate Image Saved: plates\plate_1742721756.jpg
✅ Successfully written to file: detected_plates.txt
⚠ Detected but not a number plate.
✅ Valid Plate Detected: MH 19 EQ 0001
✅ Plate Image Saved: plates\plate_1742721757.jpg
✅ Successfully written to file: detected_plates.txt
✅ Valid Plate Detected: GJ 19 EQ 0001
✅ Plate Image Saved: plates\plate_1742721758.jpg
✅ Successfully written to file: detected_plates.txt
⚠ Detected but not a number plate.
⚠ Detected but not a numb